In [1]:
import utils.eigenstate_generator as eig
import utils.cs_vqe as c
import utils.cs_vqe_ansatz as c_anz
import utils.qubit_conversion as qonvert
import ast
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from copy import deepcopy
from openfermion.linalg import LinearQubitOperator, get_sparse_operator, get_ground_state
import itertools
from statistics import median

from qiskit.circuit.parameter import Parameter
from qiskit.circuit.quantumcircuit import QuantumCircuit
from qiskit import QuantumCircuit, BasicAer, execute
from qiskit.visualization import plot_histogram
from qiskit.utils import QuantumInstance
from qiskit.aqua.components.optimizers import SLSQP
from qiskit.algorithms import VQE
from qiskit import Aer

import refactor.cs_vqe_obj as c_obj

In [2]:
import openfermion
import openfermionpyscf
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.transforms import get_fermion_operator
from openfermion.circuits import (uccsd_singlet_get_packed_amplitudes,
                               uccsd_singlet_generator, uccsd_generator,
                               uccsd_convert_amplitude_format)

singlet_bool = True # Set general UCCSD or singlet UCCSD.

bond_len = 0.772#1.45
atom_1 = 'He'
atom_2 = 'H'
basis = '3-21g'
multiplicity = 1
charge = 1

coordinate_1 = (0.0, 0.0, 0.0)
coordinate_2 = (0.0, 0.0, bond_len)
geometry = [(atom_1, coordinate_1), (atom_2, coordinate_2)]

molecule_data = MolecularData(geometry, basis, multiplicity, charge, description='Test')
#molecule.load()

# Set calculation parameters.
run_scf = 1
run_mp2 = 1
run_cisd = 0
run_ccsd = 0
run_fci = 1
delete_input = True
delete_output = True

# Run pyscf.
molecule = run_pyscf(molecule_data,
                     run_scf=run_scf,
                     run_mp2=run_mp2,
                     run_cisd=run_cisd,
                     run_ccsd=run_ccsd,
                     run_fci=run_fci)

#molecule.load()
#print(molecule)

scf = True      # Hartree-Fock.
mp2 = True      # Moller-Plesset 2.
cisd = True     # Configuration interaction singles and doubles.
ccsd = True     # Coupled cluster singles and doubles.
fci = True      # Full configuration interaction.

calculated_molecule = run_pyscf(molecule_data, scf, mp2, cisd, ccsd, fci)

if ccsd:
    ccsd_single_amps = calculated_molecule.ccsd_single_amps
    ccsd_double_amps = calculated_molecule.ccsd_double_amps

num_electrons = calculated_molecule.n_electrons
num_qubits = 2*calculated_molecule.n_orbitals

ham = get_fermion_operator(molecule.get_molecular_hamiltonian())
ham_q = jordan_wigner(ham)
ham = qonvert.QubitOperator_to_dict(ham_q, num_qubits)
print('Hamiltonian:', '\n', ham_q, '\n')

Hamiltonian: 
 (1.1324434021069392+0j) [] +
(-0.025810254414816665+0j) [X0 X1 Y2 Y3] +
(-0.012799934786436408+0j) [X0 X1 Y2 Z3 Z4 Y5] +
(-0.025159645839898638+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Y7] +
(-0.012799934786436406+0j) [X0 X1 X3 X4] +
(-0.025159645839898638+0j) [X0 X1 X3 Z4 Z5 X6] +
(-0.01711452882177433+0j) [X0 X1 Y4 Y5] +
(-0.0009891049103616359+0j) [X0 X1 Y4 Z5 Z6 Y7] +
(-0.0009891049103616357+0j) [X0 X1 X5 X6] +
(-0.05469856574684851+0j) [X0 X1 Y6 Y7] +
(0.025810254414816665+0j) [X0 Y1 Y2 X3] +
(0.012799934786436408+0j) [X0 Y1 Y2 Z3 Z4 X5] +
(0.025159645839898638+0j) [X0 Y1 Y2 Z3 Z4 Z5 Z6 X7] +
(-0.012799934786436406+0j) [X0 Y1 Y3 X4] +
(-0.025159645839898638+0j) [X0 Y1 Y3 Z4 Z5 X6] +
(0.01711452882177433+0j) [X0 Y1 Y4 X5] +
(0.0009891049103616359+0j) [X0 Y1 Y4 Z5 Z6 X7] +
(-0.0009891049103616357+0j) [X0 Y1 Y5 X6] +
(0.05469856574684851+0j) [X0 Y1 Y6 X7] +
(0.0025779051794167333+0j) [X0 Z1 X2] +
(0.004766148105680764+0j) [X0 Z1 X2 X3 Z4 X5] +
(-0.009264702923308865+0j) [X0 Z1 X2 X3 

In [3]:
terms_noncon = c.greedy_dfs(ham, 1, criterion='weight')[-1]

In [4]:
cs_vqe_object = c_obj.cs_vqe(ham, terms_noncon, num_qubits)

In [12]:
cs_vqe_object.reduced_hamiltonian(sim_qubits=8)

{'IIIIIIII': (1.1324434021069392+0j),
 'ZIIIIIZI': (-0.1399063176082599+0j),
 'YZYIIIIZ': (-0.0025779051794167333+0j),
 'XZXIIIZZ': (0.0025779051794167333+0j),
 'YZZZYIIZ': (-0.05776205910137349+0j),
 'XZZZXIZZ': (0.05776205910137349+0j),
 'IZZZZZXZ': (-0.03866070666863572+0j),
 'ZZZZZZXZ': (-0.03866070666863572+0j),
 'IZIIIIIZ': (-0.13990631760826003+0j),
 'IYZYIIII': (0.002577905179416735+0j),
 'IXZXIIIZ': (-0.002577905179416735+0j),
 'IYZZZYII': (0.05776205910137349+0j),
 'IXZZZXIZ': (-0.05776205910137349+0j),
 'IIZZZZZX': (0.03866070666863572+0j),
 'IZZZZZZX': (0.03866070666863572+0j),
 'IIZIIIII': (-0.24607368520019995+0j),
 'IIYZYIII': (0.0615554592434382+0j),
 'IIXZXIII': (0.0615554592434382+0j),
 'YIYZZZXI': (-0.020218321802526307+0j),
 'YIXZZZYI': (0.020218321802526307+0j),
 'IIIZIIII': (-0.24607368520019995+0j),
 'IIIYZYII': (0.061555459243438214+0j),
 'IIIXZXII': (0.061555459243438214+0j),
 'IYIYZZZX': (-0.02021832180252631+0j),
 'IYIXZZZY': (0.02021832180252631+0j),
 'IIIIZ